<a href="https://colab.research.google.com/github/Rehzende/sorocabatransparente/blob/main/notebooks/extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install "camelot-py[cv]" 
!pip install camelot-py[base]
!pip install "PyPDF2<3.0"
!apt install ghostscript python3-tk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.50~dfsg-5ubuntu4.6).
python3-tk is already the newest version (3.8.10-0ubuntu1~20.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [4]:
import camelot

In [5]:
tables = camelot.read_pdf('relatorio.pdf', encoding='utf-8')

In [7]:
from operator import index
df = tables[0].df 
temp_df = df.iloc[2:]
temp_df.rename(columns=temp_df.iloc[0]).drop(temp_df.index[0])
temp_df = temp_df.replace(r'\n',' ', regex=True) 
temp_df = temp_df.replace(r'^s*$', float('0'), regex = True)
temp_df = temp_df[:-2]
temp_df.columns = ['Vereador', 'Aluguel Impressora','Material Escritório','Postagem' ,'Combustivel', 'Total', 'Reembolso']



In [8]:
temp_df.reset_index(drop=True, inplace=True)
temp_df
temp_df.to_json('file.json', orient='index')

In [15]:
from PIL import Image, ImageDraw, ImageFont

def put_expenses_values(template, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total):
     fonte = ImageFont.truetype("arial.ttf", 80)
     draw = ImageDraw.Draw(template)
     draw.text((1100, 1060), f" R$ {aluguel_impressora}", font=fonte, fill="#fff")
     draw.text((1100, 1260), f" R$ {combustivel}", font=fonte, fill="#fff")
     draw.text((1100, 1460), f" R$ {mat_escritorio}", font=fonte, fill="#fff")
     draw.text((1100, 1660), f" R$ {postagem}", font=fonte, fill="#fff")
     draw.text((1100, 1860), f" R$ {reembolso}", font=fonte, fill="#fff")
     draw.text((1100, 2060), f" R$ {total}", font=fonte, fill="#000")
     return template

def put_photo_vereador(img, image_name):
    photo = Image.open(image_name)
    photo = photo.resize((947, 1433), Image.ANTIALIAS)
    posicao_photo = (1800, 800)
    img.paste(photo, posicao_photo)
    return img

def main(nome_vereador, image_name, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total):
    fonte_nome = ImageFont.truetype("arial.ttf", 80)
    img = Image.open("despesas_template.jpg")
    img = put_photo_vereador(img, image_name)
    draw = ImageDraw.Draw(img)
    draw.text((140, 800), nome_vereador, font=fonte_nome, fill="#fff")   
    img = put_expenses_values(img, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total)
    
    file_name = nome_vereador.lower().replace(" ", "_")
    img.save(f"{file_name}.jpg")


In [14]:
import json


with open('file.json') as f:
    vereadores = json.load(f)

for chave, valores in vereadores.items():
    main(valores["Vereador"], "silvano.jpg", valores["Aluguel Impressora"], valores["Material Escrit\u00f3rio"], valores["Postagem"], valores["Combustivel"], valores["Reembolso"], valores["Total"])


